In [200]:
import base64
import secrets
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad

In [201]:
def generate_random_bytes(bytes_length):
    random_bytes = secrets.token_bytes(bytes_length)
    return random_bytes

def base64decode(raw_message):  
    message = base64.b64decode(raw_message)
    return message

def encrypt_ECB(message):
    assert len(KEY) == BLOCK_LENGTH
    padded_message = pad(message + UNKNOWN_MESSAGE, BLOCK_LENGTH)
    cipher = AES.new(KEY, AES.MODE_ECB)
    encrypted_message = cipher.encrypt(padded_message) 
    return encrypted_message

# solution
def detect_block_length():
    bytes_length = 0
    empty_encrpytion_length = len(encrypt_ECB(b""))
    while bytes_length < 100:
        bytes_length += 1
        encryption_length = len(encrypt_ECB(b"a" * bytes_length))
        if encryption_length > empty_encrpytion_length:
            return encryption_length - empty_encrpytion_length

def is_ecb():
    block_len = 16 # smallest block length to start
    encrypted_message = encrypt_ECB(b"A" * 100)
    if encrypted_message[:block_len] == encrypted_message[block_len:2*block_len]:
        return "ECB"
    else:
        return "Not ECB"

In [202]:
UNKNOWN_MESSAGE_B64ENCODED = b"Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK"
UNKNOWN_MESSAGE = base64decode(UNKNOWN_MESSAGE_B64ENCODED)
BLOCK_LENGTH = 16
KEY = generate_random_bytes(BLOCK_LENGTH)

Block length

In [203]:
print("block length:", detect_block_length())
print("encryption method:", is_ecb())

block length: 16
encryption method: ECB


In [ ]:
def get_unknown_string_length():
    unknown_string_length_approx = len(encrypt_ECB(b""))
    unknown_string_length = unknown_string_length_approx
    for i in range(1, BLOCK_LENGTH):
        unknown_string_length_test = len(encrypt_ECB(b"A" * i))
        if unknown_string_length_test == unknown_string_length_approx:
            unknown_string_length -= 1
        else:
            return unknown_string_length
            

def decrypt_unknown_string():
    unknown_string_length = get_unknown_string_length()
    n_blocks = unknown_string_length // BLOCK_LENGTH 
    buffer_string = b"A" * BLOCK_LENGTH
    unknown_string = b"" 
    for i in range(n_blocks):  
        for j in range(BLOCK_LENGTH):
            message = buffer_string[j+1:]
            original_encrypted_message = encrypt_ECB(message)
            for k in range(256):
                l_unknown_string = len(unknown_string)
                test_message = message + unknown_string[l_unknown_string-j:] + bytes([k])
                if i == 1 and j == 0:
                    print(unknown_string[-j:])
                    break
                test_encypted_message =  encrypt_ECB(test_message)
                if test_encypted_message[:BLOCK_LENGTH] == original_encrypted_message[i*BLOCK_LENGTH:(i+1)*BLOCK_LENGTH]:
                    unknown_string = unknown_string + bytes([k])
                    break
        buffer_string = unknown_string[-BLOCK_LENGTH:]
    return unknown_string

In [205]:
decrypt_unknown_string()

b"Rollin' in my 5."


b"Rollin' in my 5."

In [206]:
a = b"Rollin' in my 5."

In [207]:
b = encrypt_ECB(a[1:])[BLOCK_LENGTH:2*BLOCK_LENGTH]

In [208]:
for k in range(256):
    if encrypt_ECB(a[1:]+bytes([k]))[:BLOCK_LENGTH] == b:
        print(k)

48
